In [ ]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector, DensityMatrix, Pauli, Operator, random_statevector, partial_trace
from qiskit_aer import AerSimulator, Aer
import numpy as np
from scipy.linalg import sqrtm
import cvxpy as cp
import itertools
from IPython.display import clear_output
from winsound import Beep

def tensor_prod(*tensors):
    if len(tensors) == 2:
        return np.kron(tensors[0], tensors[1])
    else:
        return np.kron(tensors[0], tensor_prod(*tensors[1:]))
    
def hermitian(matrix):
    return np.allclose(matrix, matrix.conj().T)

def trace_one(matrix):
    return np.isclose(np.trace(matrix), 1)

def positive_semi_definite(matrix, tol=1e-8):
    return np.all(np.linalg.eigvals(matrix) + tol >= 0)

def is_legal(matrix):
    return hermitian(matrix) and trace_one(matrix) and positive_semi_definite(matrix)

def check_legal(matrix, print_errors=True):
    errors, legal = [], True
    if not hermitian(matrix):
        errors.append('not hermitian')
    if not trace_one(matrix):
        errors.append('trace not equal to one')
    if not positive_semi_definite(matrix):
        errors.append('not positive semidefinite')
    if len(errors) > 0:
        legal = False
    if print_errors:
        if not legal:
            print(f'input is not legal: ' + '; '.join(errors))
        else: 
            print('input is a legal density matrix')
    return legal
        
        
def generate_prob_lst(num_states):
    prob_lst = np.array([np.random.random() for _ in range(num_states)])
    prob_lst /= np.sum(prob_lst)
    return prob_lst

def get_rank(dm, tol=1e-10):
    return int(np.sum(np.linalg.eigvalsh(dm) > tol))

def get_fidelity(dm1, dm2, tol=1e-5):
    # assert is_legal(dm1) and is_legal(dm2), 'inputs are not legal density matrices'
    if not is_legal(dm1) and is_legal(dm2):
        print("Warning: inputs are not legal density matrices")
    try: 
        fidelity = (np.trace(sqrtm(sqrtm(dm1) @ dm2 @ sqrtm(dm1)))) ** 2
    except ValueError:
        print('fidelity cannot be computed for given inputs')
    assert np.abs(np.imag(fidelity)) < tol, 'fidelity is not real within tol'
    return fidelity.real

def generate_dm(num_qubits, num_states, state_lst=None, prob_lst=None, prime_prob=None, compute_fidelity=False):
    assert (prob_lst is None) or (prime_prob is None), 'cannot set prob_lst and prime_prob together'
    if state_lst is None:
        state_lst = [random_statevector(2**num_qubits) for _ in range(num_states)]
        if compute_fidelity:
            prime_state = DensityMatrix(state_lst[0]).data
    if prime_prob is not None:
        prob_lst = np.array([prime_prob] + (generate_prob_lst(num_states - 1) * (1 - prime_prob)).tolist())
    elif prob_lst is None:
        prob_lst = generate_prob_lst(num_states)
    density_matrix = sum([DensityMatrix(state_lst[i]).data * prob_lst[i] for i in range(num_states)])
    if compute_fidelity:
        fidelity = get_fidelity(prime_state, density_matrix)
        return density_matrix, fidelity
    else:
        return density_matrix

def generate_random_Pauli_strings(num_strings, length, contain_I=True):
    if contain_I:
        characters = ['I', 'X', 'Y', 'Z']
    else:
        characters = ['X', 'Y', 'Z']
    generated_strings = []
    assert num_strings < len(characters) ** length, 'too much strings to generate'
    for _ in range(num_strings):
        while True:
            random_string = ''.join(np.random.choice(characters) for _ in range(length))
            if random_string != 'I' * length and random_string not in generated_strings:
                generated_strings.append(random_string)
                break
    return generated_strings

def generate_random_01_strings(num_strings, length):
    characters = ['0', '1']
    generated_strings = []
    assert num_strings < 2 ** length, 'too much strings to generate'
    for _ in range(num_strings):
        while True:
            random_string = ''.join(np.random.choice(characters) for _ in range(length))
            if random_string != '0' * length and random_string not in generated_strings:
                generated_strings.append(random_string)
                break
    return generated_strings

def generate_uv_Pauli_matrix(u_vec, v_vec):
    uv_map = {
        '00':'I', '01':'Z', '10':'X', '11':'Y'
    }
    Pauli_string = ''
    for u, v in zip(u_vec, v_vec):
        Pauli_char = uv_map.get(str(u) + str(v))
        if Pauli_char is None:
            raise ValueError('u or v list contains elements neither 0 or 1')
        else:
            Pauli_string += Pauli_char
    return Pauli_string

def generate_all_binary(length):
    all_strings = itertools.product('01', repeat=length)
    result = [''.join(s) for s in all_strings if '1' in s]
    return result

def generate_Pauli_expectations(dm, obsv):
    return np.trace(dm @ Pauli(obsv).to_matrix()).real

def get_trace_norm(dm):
    return np.sum(np.linalg.svd(dm, compute_uv=False))

In [ ]:
def generate_random_Pauli_strings(num_strings, num_qubits, pattern='balanced'):
    assert pattern in ['balanced', 'pro_I', 'pro_XYZ', 'uv_pair'], 'please choose pattern from: balanced, pro_I, pro_XYZ, uv_pair'
    generated_strings = []
    characters = ['X', 'Y', 'Z', 'I']
    assert 0 < num_strings <= 4 ** num_qubits - 1, 'too much or too few strings to generate'
    if pattern == 'balanced':
        for _ in range(num_strings):
            while True:
                random_string = ''.join(np.random.choice(characters) for _ in range(num_qubits))
                if random_string != 'I' * num_qubits and random_string not in generated_strings:
                    generated_strings.append(random_string)
                    break
        return generated_strings
    if pattern == 'uv_pair':
        uv_map = {'00':'I', '01':'Z', '10':'X', '11':'Y'}
        whole, remain = num_strings // 2 ** (num_qubits), num_strings % 2 ** (num_qubits)
        if whole > 0:
            v_lst = [format(i, f'0{num_qubits}b') for i in range(2 ** num_qubits)]
            u_lst = np.random.choice([format(i, f'0{num_qubits}b') for i in range(1, 2 ** num_qubits)], whole, replace=False).tolist()
            for u, v in list(itertools.product(u_lst, v_lst)):
                generated_strings.append(''.join([uv_map[u_char + v_char] for u_char, v_char in zip(u, v)]))
        if remain > 0:
            u_lst = ['0' * num_qubits]
            v_lst = np.random.choice([format(i, f'0{num_qubits}b') for i in range(1, 2 ** num_qubits)], remain, replace=False).tolist()
            for u, v in list(itertools.product(u_lst, v_lst)):
                generated_strings.append(''.join([uv_map[u_char + v_char] for u_char, v_char in zip(u, v)]))
        return generated_strings
    all_strings = generate_random_Pauli_strings(4 ** num_qubits - 1, num_qubits, pattern='balanced')
    grouped = dict()
    for i in range(num_qubits):
        grouped[i] = []
    for string in all_strings:
        grouped[string.count('I')].append(string)
    if pattern == 'pro_I':
        i = num_qubits - 1
        while len(generated_strings) < num_strings:
            if num_strings - len(generated_strings) >= len(grouped[i]):
                generated_strings += grouped[i]
            else:
                generated_strings += np.random.choice(grouped[i], num_strings - len(generated_strings), replace=True).tolist()
            i -= 1
        return generated_strings
    if pattern == 'pro_XYZ':
        i = 0
        while len(generated_strings) < num_strings:
            if num_strings - len(generated_strings) >= len(grouped[i]):
                generated_strings += grouped[i]
            else:
                generated_strings += np.random.choice(grouped[i], num_strings - len(generated_strings), replace=True).tolist()
            i += 1
        return generated_strings
        

In [ ]:
def estimate_Pauli_expectations(dm, obsv, num_samples, simulation=False):
    num_samples = int(num_samples)
    if simulation: # simulate the process of sampling
        exp = np.real(np.trace(dm @ Pauli(obsv).to_matrix()))
        prob_p1 = (1 + exp) / 2
        prob_m1 = 1 - prob_p1
        samples = np.random.choice([+1, -1], size=num_samples, p=[prob_p1, prob_m1])
        return np.mean(samples)
    else: # use the approximate distribution instead
        exp = np.real(np.trace(dm @ Pauli(obsv).to_matrix()))
        num_samples_root = num_samples ** .5
        std_dev = (1 - exp ** 2) ** .5 / num_samples_root
        return np.random.normal(exp, std_dev)
    
def get_partial_trace(dm, subsystems):
    return partial_trace(dm, subsystems).data

def get_von_neumann_entropy(dm, r=None):
    # if not np.allclose(dm, dm.conj().T):
    #     raise ValueError("The density matrix must be Hermitian.")
    if r is None:
        eigenvalues = np.linalg.eigvalsh(dm)
        eigenvalues = eigenvalues[eigenvalues > 0]
        entropy = - np.sum(eigenvalues * np.log2(eigenvalues))
    else:
        eigenvalues = np.linalg.eigvalsh(dm)
        eigenvalues = np.partition(eigenvalues, -r)[-r:]
        entropy = - np.sum(eigenvalues * np.log2(eigenvalues))
    return entropy

def get_TMI(dm, r=None):
    dm_s = get_partial_trace(dm, [1, 2])
    dm_m1 = get_partial_trace(dm, [0, 2])
    dm_m2 = get_partial_trace(dm, [1, 2])
    dm_m = get_partial_trace(dm, [0])
    dm_sm1 = get_partial_trace(dm, [2])
    dm_sm2 = get_partial_trace(dm, [1])
    i2_sm1 = get_von_neumann_entropy(dm_s, r) + get_von_neumann_entropy(dm_m1, r) - get_von_neumann_entropy(dm_sm1, r)
    i2_sm2 = get_von_neumann_entropy(dm_s, r) + get_von_neumann_entropy(dm_m2, r) - get_von_neumann_entropy(dm_sm2, r)
    i2_sm = get_von_neumann_entropy(dm_s, r) + get_von_neumann_entropy(dm_m, r) - get_von_neumann_entropy(dm, r)
    return i2_sm1 + i2_sm2 - i2_sm

def regularize(dm):
    dm /= np.trace(dm)
    dm = (dm + dm.conj().T) / 2
    return dm


In [ ]:
def trial(dm, num_qubits, obsv, expct, tol=1e-5):
    true_TMI = get_TMI(dm)
    while not np.all(np.array([np.abs(np.trace(dm @ Pauli(o).to_matrix()) - e) <= 1e-5 for o, e in zip(obsv, expct)])):
        tol *= 10 ** .25
    def optimize(dim, obsv, expct, tol=1e-5):
        sigma = cp.Variable((dim, dim), complex=True)
        objective = cp.Minimize(cp.abs(5 * cp.norm(sigma, 'nuc') + 0 * cp.norm(sigma, 'fro') ** 2))
        constraints = [cp.trace(sigma) == 1]
        for o, e in zip(obsv, expct):
            constraints.append(cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) <= tol)
        problem = cp.Problem(objective, constraints)
        problem.solve()
        if problem.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
            raise ValueError(f"Optimization failed with {len(obsv)} observables")
        return sigma.value
    try: 
        sigma = optimize(2 ** num_qubits, obsv, expct)
        sigma = regularize(sigma)
    except ValueError:
        estm_TMI = 0
    estm_TMI = get_TMI(sigma)
    err = (estm_TMI - true_TMI) / true_TMI
    return estm_TMI, err

In [ ]:
def trial(dm, num_qubits, obsv, expct, tol=1e-5):
    true_TMI = get_TMI(dm)
    while not np.all(np.array([np.abs(np.trace(dm @ Pauli(o).to_matrix()) - e) <= 1e-5 for o, e in zip(obsv, expct)])):
        tol *= 10 ** .25
    def optimize(dim, obsv, expct, tol=1e-5):
        sigma = cp.Variable((dim, dim), complex=True)
        objective = cp.Minimize(cp.sum([cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) ** 2 for o, e in zip(obsv, expct)]) + len(obsv) * cp.norm(sigma, 'nuc'))
        constraints = [sigma >> 0, cp.trace(sigma) == 1]
        problem = cp.Problem(objective, constraints)
        problem.solve()
        if problem.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
            raise ValueError(f"Optimization failed with {len(obsv)} observables")
        return sigma.value
    try: 
        sigma = optimize(2 ** num_qubits, obsv, expct)
        sigma = regularize(sigma)
    except ValueError:
        estm_TMI = 0
    estm_TMI = get_TMI(sigma)
    err = (estm_TMI - true_TMI) / true_TMI
    return estm_TMI, err

In [ ]:
num_qubits = 3
num_samples = 1e12
pmax = np.array([.90, .93, .96, .99])
num_observables = np.arange(24, 48, 2)
states_fidelities = [generate_dm(num_qubits, 5, prime_prob=p, compute_fidelity=True) for p in pmax]
states, fidelities = [s_f[0] for s_f in states_fidelities], [s_f[1] for s_f in states_fidelities]
all_observables = [generate_random_Pauli_strings(num_obs, num_qubits, contain_I=True) for num_obs in num_observables]
TMIs = [get_TMI(s) for s in states]
all_data = dict()
for i in range(4):
    state, key = states[i], str(i)
    all_data[key] = {'TMI' : [], 'err' : []}
    all_expectations = [[estimate_Pauli_expectations(state, obsv, num_samples) for obsv in observables] for observables in all_observables]
    for j in range(len(num_observables)):
        num_obs = num_observables[j]
        observables = all_observables[j]
        expectations = all_expectations[j]
        estm_TMIs, errs = [], []
        display(f"now experimenting: index {i}, measurements {num_obs}")
        for _ in range(50):
            estm_TMI, err = trial(state, num_qubits, observables, expectations, tol=1e-6)
            estm_TMIs.append(estm_TMI)
            errs.append(err)
        all_data[key]['TMI'].append(np.mean(estm_TMIs))
        all_data[key]['err'].append(np.mean(errs))
        clear_output()

In [ ]:
import json

num_qubits = 3
num_samples = 1e12
nums_observables = np.arange(25, 64, 2)
pmax = np.array([.90, .93, .96])
pmax_label=['90', '93', '96']
for i in range(3):
    p, plabel = pmax[i], pmax_label[i]
    errs_data = dict()
    for num_observables in nums_observables:
        errs = []
        for _ in range(50):
            state = generate_dm(3, 5, prime_prob=p)
            observables = generate_random_Pauli_strings(num_observables, 3, contain_I=True)
            expectations = [estimate_Pauli_expectations(state, obsv, num_samples) for obsv in observables]
            estm_TMI, err = trial(state, num_qubits, observables, expectations, tol=1e-7)
            errs.append(err)
        errs_data[int(num_observables)] = errs
    name = 'tracenorm_balanced_' + plabel + '.json'
    with open(name, 'w') as file:
        json.dump(errs_data, file, indent=4)

In [ ]:
import json

num_qubits = 3
num_samples = 1e12
nums_observables = np.arange(25, 64, 2)
pmax = np.array([.99])
pmax_label=['99']
patterns = ['balanced']
for pattern in patterns:
    for i in range(4):
        p, plabel = pmax[i], pmax_label[i]
        errs_data = dict()
        for num_observables in nums_observables:
            errs = []
            display(f"now computing: plabel={plabel}, pattern={pattern}, num_observables={num_observables}")
            for _ in range(50):
                state = generate_dm(3, 5, prime_prob=p)
                observables = generate_random_Pauli_strings(num_observables, 3, pattern)
                expectations = [estimate_Pauli_expectations(state, obsv, num_samples) for obsv in observables]
                estm_TMI, err = trial(state, num_qubits, observables, expectations, tol=1e-7)
                errs.append(err)
            errs_data[int(num_observables)] = errs
            clear_output()
        name = 'tracenorm_' + pattern + '_' + plabel + '.json'
        with open(name, 'w') as file:
            json.dump(errs_data, file, indent=4)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

patterns = ['balanced', 'pro_I', 'pro_XYZ', 'uv_pair']
methods = ['tracenorm']

fig, axs = plt.subplots(2, 2, figsize=(16, 12))


for method in methods:
    for pattern in patterns:
        path = r'von Neumann Entropy\output\fig01\\' + method + '_' + pattern + '_99.json'
        with open(path, 'r') as file:
            data = json.load(file)
        nums_observables, means, lower_bounds, upper_bounds = [], [], [], []
        for num_observables, errors in data.items():
            if int(num_observables) > 37:
                nums_observables.append(int(num_observables))
                errors = np.abs(np.array(errors))
                lower_bound = np.percentile(errors, 0)  
                upper_bound = np.percentile(errors, 95) 
                filtered_array = errors[(errors >= lower_bound) & (errors <= upper_bound)]
                means.append(np.mean(filtered_array))
                lower_bounds.append(lower_bound)
                upper_bounds.append(upper_bound)
        means = np.array(means)
        lower_bounds = np.array(lower_bounds)
        upper_bounds = np.array(upper_bounds)
        error_bars = np.vstack((means - lower_bounds, upper_bounds - means))

        axs[0, 0].plot(nums_observables, means, linewidth=1, label=pattern)
        # plt.errorbar(nums_observables, means, yerr=error_bars, fmt='o', capsize=2, capthick=1)
    
axs[0, 0].set_xlabel('Number of Observables')
axs[0, 0].set_ylabel('Mean Error (' + r'$95\%$' + 'Range)')
axs[0, 0].set_ylim([0, .35])
axs[0, 0].set_title('Mean Error for State with ' + r'$p_m=0.99$')
axs[0, 0].legend()
axs[0, 0].grid(True)

for method in methods:
    for pattern in patterns:
        path = r'von Neumann Entropy\output\fig01\\' + method + '_' + pattern + '_96.json'
        with open(path, 'r') as file:
            data = json.load(file)
        nums_observables, means, lower_bounds, upper_bounds = [], [], [], []
        for num_observables, errors in data.items():
            if int(num_observables) > 37:
                nums_observables.append(int(num_observables))
                errors = np.abs(np.array(errors))
                lower_bound = np.percentile(errors, 0)  
                upper_bound = np.percentile(errors, 95) 
                filtered_array = errors[(errors >= lower_bound) & (errors <= upper_bound)]
                means.append(np.mean(filtered_array))
                lower_bounds.append(lower_bound)
                upper_bounds.append(upper_bound)
        means = np.array(means)
        lower_bounds = np.array(lower_bounds)
        upper_bounds = np.array(upper_bounds)
        error_bars = np.vstack((means - lower_bounds, upper_bounds - means))

        axs[0, 1].plot(nums_observables, means, linewidth=1, label=pattern)
        # plt.errorbar(nums_observables, means, yerr=error_bars, fmt='o', capsize=2, capthick=1)
    
axs[0, 1].set_xlabel('Number of Observables')
axs[0, 1].set_ylabel('Mean Error (' + r'$95\%$' + 'Range)')
axs[0, 1].set_ylim([0, .35])
axs[0, 1].set_title('Mean Error for State with ' + r'$p_m=0.96$')
axs[0, 1].legend()
axs[0, 1].grid(True)

for method in methods:
    for pattern in patterns:
        path = r'von Neumann Entropy\output\fig01\\' + method + '_' + pattern + '_93.json'
        with open(path, 'r') as file:
            data = json.load(file)
        nums_observables, means, lower_bounds, upper_bounds = [], [], [], []
        for num_observables, errors in data.items():
            if int(num_observables) > 37:
                nums_observables.append(int(num_observables))
                errors = np.abs(np.array(errors))
                lower_bound = np.percentile(errors, 0)  
                upper_bound = np.percentile(errors, 95) 
                filtered_array = errors[(errors >= lower_bound) & (errors <= upper_bound)]
                means.append(np.mean(filtered_array))
                lower_bounds.append(lower_bound)
                upper_bounds.append(upper_bound)
        means = np.array(means)
        lower_bounds = np.array(lower_bounds)
        upper_bounds = np.array(upper_bounds)
        error_bars = np.vstack((means - lower_bounds, upper_bounds - means))

        axs[1, 0].plot(nums_observables, means, linewidth=1, label=pattern)
        # plt.errorbar(nums_observables, means, yerr=error_bars, fmt='o', capsize=2, capthick=1)
    
axs[1, 0].set_xlabel('Number of Observables')
axs[1, 0].set_ylabel('Mean Error (' + r'$95\%$' + 'Range)')
axs[1, 0].set_ylim([0, .35])
axs[1, 0].set_title('Mean Error for State with ' + r'$p_m=0.93$')
axs[1, 0].legend()
axs[1, 0].grid(True)

for method in methods:
    for pattern in patterns:
        path = r'von Neumann Entropy\output\fig01\\' + method + '_' + pattern + '_90.json'
        with open(path, 'r') as file:
            data = json.load(file)
        nums_observables, means, lower_bounds, upper_bounds = [], [], [], []
        for num_observables, errors in data.items():
            if int(num_observables) > 37:
                nums_observables.append(int(num_observables))
                errors = np.abs(np.array(errors))
                lower_bound = np.percentile(errors, 0)  
                upper_bound = np.percentile(errors, 95) 
                filtered_array = errors[(errors >= lower_bound) & (errors <= upper_bound)]
                means.append(np.mean(filtered_array))
                lower_bounds.append(lower_bound)
                upper_bounds.append(upper_bound)
        means = np.array(means)
        lower_bounds = np.array(lower_bounds)
        upper_bounds = np.array(upper_bounds)
        error_bars = np.vstack((means - lower_bounds, upper_bounds - means))

        axs[1, 1].plot(nums_observables, means, linewidth=1, label=pattern)
        # plt.errorbar(nums_observables, means, yerr=error_bars, fmt='o', capsize=2, capthick=1)
    
axs[1, 1].set_xlabel('Number of Observables')
axs[1, 1].set_ylabel('Mean Error (' + r'$95\%$' + 'Range)')
axs[1, 1].set_ylim([0, .35])
axs[1, 1].set_title('Mean Error for State with ' + r'$p_m=0.90$')
axs[1, 1].legend()
axs[1, 1].grid(True)
fig.suptitle('Mean Error for Nearly Pure States, $r=5$, with $10^{12}$ samples', fontsize=24)
plt.subplots_adjust(top=.92)
plt.savefig('fig01.png', dpi=1200)
plt.show()


In [ ]:
errs = []
for _ in range(100):
    state = generate_dm(3, 1, prime_prob=1)
    observables = generate_random_Pauli_strings(49, 3, 'uv_pair')
    expectations = [estimate_Pauli_expectations(state, obsv, num_samples) for obsv in observables]
    estm_TMI, err = trial(state, num_qubits, observables, expectations, tol=1e-7)
    errs.append(abs(err))
errs = np.array(errs)
# upper_bound = np.percentile(errors, 95)
# errs = errs[errs < upper_bound]
np.mean(errs)

In [ ]:
errors = dict()
ranks = np.array([1, 2, 3, 4, 5, 6, 7, 8])
for r in ranks:
    errs = []
    if r == 1:
        pmax = 1
    else:
        pamx = .93
    for _ in range(100):
        state = generate_dm(3, r, prime_prob=pmax)
        observables = generate_random_Pauli_strings(49, 3, 'uv_pair')
        expectations = [estimate_Pauli_expectations(state, obsv, num_samples) for obsv in observables]
        estm_TMI, err = trial(state, num_qubits, observables, expectations, tol=1e-7)
        errs.append(abs(err))
    errs = np.array(errs)
    # upper_bound = np.percentile(errs, 95)
    # errs = errs[errs < upper_bound]
    errors[str(r)] = errs

In [ ]:
mean_data, std_dev_data = [], []
for errs in errors.values():
    upper_bound = np.percentile(errs, 95)
    errs = errs[errs < upper_bound]
    mean_data.append(np.mean(errs))
    std_dev_data.append(np.std(errs))
mean_data = np.array(mean_data)
std_dev_data = np.array(std_dev_data)
plt.plot(ranks, mean_data, linewidth=2, color='orange')
plt.errorbar(ranks, mean_data, yerr=std_dev_data, color='red', fmt='o', label='Data with std dev error bars')
plt.xlabel('rank')
plt.ylabel('Mean Error (' + r'$95\%$' + 'Range)')
plt.title('Mean Error for $p_m=0.93$ states, with $10^{12}$ samples')
plt.savefig('fig02.png', dpi=1200)
plt.show()

In [ ]:
table = np.zeros([3, 3])
num_samples = 1e12
num_qubits = 3
i, j, k = 0, 0, 0
while True:
    state = generate_dm(3, 3, prime_prob=.99)
    observables = generate_random_Pauli_strings(49, 3, 'uv_pair')
    expectations = [estimate_Pauli_expectations(state, obsv, num_samples) for obsv in observables]
    estm_TMI, err = trial(state, num_qubits, observables, expectations, tol=1e-7)
    if abs(estm_TMI) < .01:
        i += 1
        if i <= 1000:
            if err < .02:
                table[0, 0] += 1
            elif .02 <= err <= .2:
                table[0, 1] += 1
            else:
                table[0, 2] += 1
    elif .01 <= abs(estm_TMI) <= .05:
        j += 1
        if j <= 1000:
            if err < .02:
                table[1, 0] += 1
            elif .02 <= err <= .2:
                table[1, 1] += 1
            else:
                table[1, 2] += 1
    else:
        k += 1
        if k <= 1000:
            if err < .02:
                table[2, 0] += 1
            elif .02 <= err <= .2:
                table[2, 1] += 1
            else:
                table[2, 2] += 1
    if i >= 1000 and j >= 1000 and k >= 1000:
        break


In [ ]:
table